In [15]:
import os
os.chdir("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/")

from math import e
from re import L
import numpy
import torch
import os.path as osp
from typing import List, Optional
import torch.nn.functional as F
from torch.nn.utils import clip_grad_value_
import pandas
from torch.nn import MSELoss, CrossEntropyLoss
from torch.nn import Module, ModuleList, Sequential, Linear, ReLU, SELU
from torch_geometric.loader import DataLoader
from torch_geometric.nn.inits import reset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import knn_graph, ChebConv, GINConv, GraphNorm, GAE, VGAE
from sklearn.metrics.cluster import adjusted_rand_score
from scripts.CustomFunc import onehot_to_label, sqrtm
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse, negative_sampling
from torch_geometric.nn.dense.mincut_pool import _rank3_trace
from scripts.SGEDataset import SGEDataset
from scripts.SGWTConv import SGWTConv
from scripts.SGEModel import ResNet, PPIEdgeConv, DynamicEdgeConv
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import adjusted_rand_score
import os

EPS = 1e-15
MAX_LOGSTD = 10
MAX = 10.
MIN = -10.



In [16]:
import pandas
import torch
import os.path as osp
from typing import List
from torch_geometric.data import InMemoryDataset, download_url, Data
from torch_geometric.utils import dense_to_sparse, remove_self_loops, to_undirected
from sklearn.metrics import pairwise_distances

#generate dataset from whole-genome HiC data
#SGE2 = SGEDataset(root='/public/home/qiyuan/Projects/Spatial/', name='SGE2', id=sample_id)
class SGEDataset1(InMemoryDataset):
    url = ''
    def __init__(self, root: str, name: str, id: List[str], transform=None, pre_transform=None):
        self.name = name
        self.id = id
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    @property
    def raw_dir(self) -> str:
        return osp.join(self.root, self.name, 'raw')    
    @property
    def processed_dir(self) -> str:
        return osp.join(self.root, self.name, 'processed1')
    @property
    def raw_file_names(self) -> List[str]:         
        files = ['hvg_counts.txt', 'truth.txt']    
        return [f'{i}_{f}' for i in self.id for f in files]  
    @property
    def processed_file_names(self) -> str:
        return 'data.pt'
    def process(self):
        data_list = []
        for i in self.id:
            print(f'sample ID: {i}')
            nodes = pandas.read_csv(f'{self.raw_dir}/{i}_hvg_counts.txt', sep=' ', header=0,index_col=0)
            x = torch.tensor(nodes.values, dtype=torch.float).transpose(0, 1)
            node_id = nodes.columns.values
            node_label = pandas.read_csv(f'{self.raw_dir}/{i}_truth.txt', sep=' ', header=0,index_col=0) 
            print(node_label) 
            node_barcode = node_label['barcode'].values
            print(node_barcode)
            if not all(node_id == node_barcode):
                try:
                    raise Exception('Node IDs do not match!')
                except Exception as error:
                    print(error) 
            node_celltype = node_label['layer'].values.tolist()
            print(node_celltype)
            y = torch.tensor(node_celltype, dtype=torch.float32)
            j = torch.where(~torch.isnan(y))    
            edge_index = torch.Tensor(2, 0) #empty edges    
            pos = pandas.read_csv(f'{self.raw_dir}/{i}_position.txt', sep=' ', header=0,index_col=0) 
            print(pos)
            # coord = torch.tensor(pos.iloc[:,-2:].values, dtype=torch.float32) 
            coord = torch.tensor(pos.iloc[:,[4, 5]].values, dtype=torch.float32)   
            dst = torch.tensor(pairwise_distances(coord[j], metric='euclidean'))
            # print(dst)
            cut = torch.sort(dst[:,100])[0][6]+5
            adj = (dst < cut).int()
            edge_index, _ = dense_to_sparse(adj)
            edge_index, _ = remove_self_loops(edge_index)
            edge_index = to_undirected(edge_index)
            edge_weight = torch.ones_like(edge_index[0])
            batch = torch.zeros(x.size(0))
            g = Data(x=x[j], y=y[j].long(), edge_index=edge_index, edge_weight=edge_weight,batch = batch)
            data_list = data_list + [g]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.set_device(1)

In [4]:
def create_mask(num_nodes, train_ratio=0.5):
    mask = torch.rand(num_nodes) < train_ratio
    return mask

In [5]:
import pandas
import torch
import os.path as osp
from typing import List
from torch_geometric.data import InMemoryDataset, download_url, Data
from torch_geometric.utils import dense_to_sparse, remove_self_loops, to_undirected
from sklearn.metrics import pairwise_distances

#generate dataset from whole-genome HiC data
#SGE2 = SGEDataset(root='/public/home/qiyuan/Projects/Spatial/', name='SGE2', id=sample_id)
def SGEDataset(id: str, raw_dir: str):
    data_list = []
    for i in id:
        print(f'sample ID: {i}')
        nodes = pandas.read_csv(f'{raw_dir}/{i}_hvg_counts.txt', sep=' ', header=0,index_col=0)
        x = torch.tensor(nodes.values, dtype=torch.float).transpose(0, 1)
        node_id = nodes.columns.values
        node_label = pandas.read_csv(f'{raw_dir}/{i}_truth.txt', sep=' ', header=0,index_col=0) 
        # print(node_label) 
        node_barcode = node_label['barcode'].values
        # print(node_barcode)
        if not all(node_id == node_barcode):
            try:
                raise Exception('Node IDs do not match!')
            except Exception as error:
                print(error) 
        node_celltype = node_label['layer'].values.tolist()
        # print(node_celltype)
        y = torch.tensor(node_celltype, dtype=torch.float32)
        j = torch.where(~torch.isnan(y))    
        num_nodes = len(y[j])
        train_mask = create_mask(num_nodes)
        test_mask = ~train_mask
        edge_index = torch.Tensor(2, 0) #empty edges    
        pos = pandas.read_csv(f'{raw_dir}/{i}_position.txt', sep=' ', header=0,index_col=0) 
        # print(pos)
        # coord = torch.tensor(pos.iloc[:,-2:].values, dtype=torch.float32) 
        coord = torch.tensor(pos.iloc[:,[4, 5]].values, dtype=torch.float32)   
        dst = torch.tensor(pairwise_distances(coord[j], metric='euclidean'))
        # print(dst)
        cut = torch.sort(dst[:,100])[0][6]+5
        adj = (dst < cut).int()
        adj1 = adj[train_mask]
        adj1 = adj1[:,train_mask]
        adj2 = adj[test_mask]
        adj2 = adj2[:,test_mask]
        edge_index1, _ = dense_to_sparse(adj1)
        edge_index1, _ = remove_self_loops(edge_index1)
        edge_index1 = to_undirected(edge_index1)
        edge_index2, _ = dense_to_sparse(adj2)
        edge_index2, _ = remove_self_loops(edge_index2)
        edge_index2 = to_undirected(edge_index2)
        edge_weight1 = torch.ones_like(edge_index1[0])
        edge_weight2 = torch.ones_like(edge_index2[0])
        batch1 = torch.zeros(adj1.size(0)).to(device)
        batch2 = torch.zeros(adj2.size(0)).to(device)
        g = Data(x=x[j][train_mask], y=y[j][train_mask].long(), edge_index=edge_index1, edge_weight=edge_weight1,batch = batch1)
        g1 = Data(x=x[j][test_mask], y=y[j][test_mask].long(), edge_index=edge_index2, edge_weight=edge_weight2,batch = batch2)
        data_list = [g,g1]
        return data_list

In [141]:
raw_dir='/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/raw'
id="151676"

nodes = pandas.read_csv(f'{raw_dir}/{id}_hvg_counts.txt', sep=' ', header=0,index_col=0)
x = torch.tensor(nodes.values, dtype=torch.float).transpose(0, 1)
node_id = nodes.columns.values
node_label = pandas.read_csv(f'{raw_dir}/{id}_truth.txt', sep=' ', header=0,index_col=0) 
# print(node_label) 
node_barcode = node_label['barcode'].values
# print(node_barcode)
if not all(node_id == node_barcode):
    try:
        raise Exception('Node IDs do not match!')
    except Exception as error:
        print(error) 
node_celltype = node_label['layer'].values.tolist()
# print(node_celltype)
y = torch.tensor(node_celltype, dtype=torch.float32)
j = torch.where(~torch.isnan(y)) 
num_nodes = len(y[j])
train_mask = create_mask(num_nodes)
test_mask = ~train_mask
edge_index = torch.Tensor(2, 0) #empty edges    
pos = pandas.read_csv(f'{raw_dir}/{id}_position.txt', sep=' ', header=0,index_col=0) 
# print(pos)
# coord = torch.tensor(pos.iloc[:,-2:].values, dtype=torch.float32) 
coord = torch.tensor(pos.iloc[:,[4, 5]].values, dtype=torch.float32)   
dst = torch.tensor(pairwise_distances(coord[j], metric='euclidean'))
# print(dst)
cut = torch.sort(dst[:,100])[0][6]+5
adj = (dst < cut).int()
adj1 = adj[train_mask]
adj1 = adj1[:,train_mask]
adj2 = adj[test_mask]
adj2 = adj2[:,test_mask]
print(adj1)
edge_index1, _ = dense_to_sparse(adj1)
edge_index1, _ = remove_self_loops(edge_index1)
edge_index1 = to_undirected(edge_index1)
edge_index2, _ = dense_to_sparse(adj2)
edge_index2, _ = remove_self_loops(edge_index2)
edge_index2 = to_undirected(edge_index2)
edge_weight1 = torch.ones_like(edge_index1[0])
edge_weight2 = torch.ones_like(edge_index2[0])
batch = torch.zeros(x.size(0))
g = Data(x=x[j][train_mask], y=y[j][train_mask].long(), edge_index=edge_index1, edge_weight=edge_weight1,batch = batch)
g1 = Data(x=x[j][test_mask], y=y[j][test_mask].long(), edge_index=edge_index2, edge_weight=edge_weight2,batch = batch)
data_list = [g,g1]

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], dtype=torch.int32)


In [6]:
sample_id = ["151507"]
d = SGEDataset(raw_dir='/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/raw',id=sample_id)

sample ID: 151507


In [17]:
EPS = 1e-15
sample_id = list(range(151507, 151511)) + list(range(151669, 151677))
SGE_train = SGEDataset1(root='/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/', name='SGE2', id=sample_id)
SGE_test = SGEDataset1(root='/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/', name='SGE2', id=sample_id)
train_loader = DataLoader(SGE_train[0,2,4,6,8,10], batch_size=1, shuffle=True)
test_loader = DataLoader(SGE_train[1,3,5,7,9,11], batch_size=1, shuffle=False)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# torch.cuda.set_device(1)

In [7]:
def create_mask(num_nodes, train_ratio=0.8):
    mask = torch.rand(num_nodes) < train_ratio
    return mask

data = data.to(device)  # 假设这是你的图数据
num_nodes = data.x.size(0)

train_mask = create_mask(num_nodes).to(device)
test_mask = ~train_mask

train_data = data.clone()  # 创建数据的副本
test_data = data.clone()

# # 更新节点信息
train_data.x = data.x[train_mask]
test_data.x = data.x[test_mask]

# # 更新边信息
train_edge_mask = train_mask[data.edge_index[0]] & train_mask[data.edge_index[1]]
test_edge_mask = test_mask[data.edge_index[0]] & test_mask[data.edge_index[1]]

train_data.edge_index = data.edge_index[:, train_edge_mask]
test_data.edge_index = data.edge_index[:, test_edge_mask]

# # 根据需要更新其他属性
train_data.x= data.x
test_data.x = data.x

In [38]:
con = torch.Tensor(2, 0) 

In [18]:
Nv, N = SGE_train[0].x.size() 
# Nv, N = d[0].x.size() 
# Nv2, N = SGE_train[0][1].x.size()   
# ppi = pandas.read_csv('/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/GBM/PPI.connect.txt', sep=' ', header=0)
ppi = pandas.read_csv('/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/PPI.connect.txt', sep=' ', header=None)
con = ppi.values[:, range(1,3)].transpose()
con = torch.tensor(con.astype(int), dtype=torch.long)
M = con.shape[1]
# emd = pandas.read_csv('/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/emd_8_samples.txt',sep = '\t', header=0)
# emd = torch.tensor(emd.values)[:, [6, 0, 1, 2, 3, 4, 5]]
# emd = emd[[6, 0, 1, 2, 3, 4, 5]]
# #K_empiric = torch.exp(-5E2 * torch.log(emd / 4))
# K_empiric = torch.exp(-18.5 * torch.log(emd / 4))
# K_sparse = dense_to_sparse(K_empiric)
# edge_pos = K_sparse[0][:, K_sparse[1] > .5]
# edge_pos = edge_pos.to(device)
# edge_neg = K_sparse[0][:, K_sparse[1] < .5]
# edge_neg = edge_neg.to(device)
# edge_weight = K_sparse[1][K_sparse[1] > .5]
# edge_weight = edge_weight.to(device)
con1 = torch.tensor([numpy.arange(0,N),numpy.arange(0,N)])
con = torch.cat([con,con1],dim = 1)


In [19]:
from math import e
import numpy
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.kl import kl_divergence
import torch.nn.functional as F
from torch.nn import Sequential, Linear, Dropout, ReLU, LeakyReLU, Softmax, Parameter
from sparselinear import SparseLinear
from torch_geometric.nn import MessagePassing, knn_graph, DMoNPooling, GINConv, EdgePooling, GraphNorm, MessageNorm
from torch_geometric.utils import erdos_renyi_graph, unbatch, to_dense_adj, to_dense_batch, dense_to_sparse, remove_self_loops, add_remaining_self_loops
from scripts.sqrtm import sqrtm
from scripts.CustomFunc import scale, dropout_edge
from scripts.SGWTConv import SGWTConv


class ResNet(torch.nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module
    def forward(self, inputs):
        return self.module(inputs) + inputs

'''
class ResNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.module = Sequential(
                    Linear(in_channels, hidden_channels),
                    LeakyReLU(0.1),
                    Linear(hidden_channels, in_channels)
                ) 
    def forward(self, inputs):
        return self.module(inputs) + inputs
'''

class FixedSparseEdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels, connect):
        super().__init__(aggr='max') #  "Maximum" aggregation.
        self.out_dim = out_channels
        self.lin1 = SparseLinear(2 * in_channels, in_channels, connectivity=connect)
        self.lin2 = Linear(in_channels, out_channels)
    def forward(self, x, edge_index, edge_weight):
        return self.propagate(edge_index, x=x, edge_weight=edge_weight)
    def message(self, x_i, x_j, edge_weight):
        out = torch.cat([x_i, x_j], dim=1) 
        # print(out.shape)
        out = F.relu(self.lin1(out)) + x_i
        out = self.lin2(out)
        # print(out)
        # print(out.shape)
        n_h = edge_weight.size(1)   
        s_h = self.out_dim // n_h   
        out = torch.matmul(out.view(-1, s_h, n_h), edge_weight) 
        # print(out.shape)
        # print(out.view(-1, n_h * s_h).shape)
        return out.view(-1, n_h * s_h)

class FullEdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='mean') #  "Mean" aggregation.
        self.out_dim = out_channels
        self.lin1 = Linear(2 * in_channels, in_channels)
        self.lin2 = Linear(in_channels, out_channels)
    def forward(self, x, edge_index, edge_weight):
        return self.propagate(edge_index, x=x, edge_weight=edge_weight)
    def message(self, x_i, x_j, edge_weight):
        out = torch.cat([x_i, x_j], dim=1) 
        out = F.relu(self.lin1(out)) + x_i
        out = self.lin2(out)
        n_h = edge_weight.size(1)   
        s_h = self.out_dim // n_h   
        out = torch.matmul(out.view(-1, s_h, n_h), edge_weight) 
        return out.view(-1, n_h * s_h)

class PPIEdgeConv(FixedSparseEdgeConv):
    def __init__(self, in_channels, out_channels, connect, pi=None, dropout=None, k=None, n_heads=2):
        super().__init__(in_channels, out_channels, connect)
        self.pi = pi
        self.dropout = dropout
        self.k = k
        self.n_heads = n_heads
        self.head_size = out_channels // n_heads
        self.d_k = out_channels
        self.qnet = Sequential(
                        SparseLinear(2 * in_channels, in_channels, connectivity=connect),
                        Linear(in_channels, out_channels),
                    )
        self.knet = Sequential(
                        SparseLinear(2 * in_channels, in_channels, connectivity=connect),
                        Linear(in_channels, out_channels),
                    )
    def reshape(self, e):
        new_shape = e.size()[:-1] + (self.n_heads, self.head_size)
        e = e.view(*new_shape)
        return e
    def forward(self, x, edge_index, batch=None):    
        if self.k is None:
            n = x.size(0)  
            edge_index = torch.zeros(2, 0, dtype=torch.long, device=x.device)
            for v in unbatch(torch.arange(n, device=x.device), batch):
                e = erdos_renyi_graph(n, self.pi, directed=False)
                e = e.to(x.device)
                edge_index = torch.cat([edge_index, v[e]], dim=1)                   
        if self.pi is None:
            edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)    
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = dropout_edge(edge_index, self.dropout, force_undirected=True)
        x_src, x_dst = x[edge_index]
        Q = self.qnet(torch.cat([x_src, x_dst], dim=1))
        K = self.knet(torch.cat([x_src, x_dst], dim=1))
        Q = self.reshape(Q)
        K = self.reshape(K)
        edge_weight = torch.matmul(Q, K.permute(0, 2, 1)) / torch.tensor(self.d_k).sqrt()
        return super().forward(x, edge_index, edge_weight)

class DynamicEdgeConv(FullEdgeConv):
    def __init__(self, in_channels, out_channels, k=4, dropout=0.0, n_heads=2):
        super().__init__(in_channels, out_channels)
        self.k = k
        self.dropout = dropout
        self.n_heads = n_heads
        self.head_size = out_channels // n_heads
        self.d_k = out_channels
        self.qnet = Linear(2 * in_channels, out_channels)
        self.knet = Linear(2 * in_channels, out_channels)
    def reshape(self, e):
        new_shape = e.size()[:-1] + (self.n_heads, self.head_size)
        e = e.view(*new_shape)
        return e    
    def forward(self, x, edge_index, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        edge_index, _ = dropout_edge(edge_index, self.dropout, force_undirected=True)
        x_src, x_dst = x[edge_index]
        Q = self.qnet(torch.cat([x_src, x_dst], dim=1)) 
        K = self.knet(torch.cat([x_src, x_dst], dim=1))
        Q = self.reshape(Q)
        K = self.reshape(K)
        edge_weight = torch.matmul(Q, K.permute(0, 2, 1)) / torch.tensor(self.d_k).sqrt()
        return super().forward(x, edge_index, edge_weight)


In [20]:
class STx_encoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, m, l, connect, pi, n_heads, K):
        super().__init__()
        torch.manual_seed(12345)
        self.n_heads = n_heads
        self.conv0 = SGWTConv(in_channels, hidden_channels, K)
        self.norm1 = GraphNorm(hidden_channels)
        self.conv1 = SGWTConv(in_channels + hidden_channels, hidden_channels, K)
        self.fc1 = Sequential(
                            Linear(hidden_channels, m * 16),
                            SELU(),
                            Linear(m * 16, m * 4),
                            SELU(),
                            Linear(m * 4, m)
                            )
        self.conv2 = PPIEdgeConv(in_channels, hidden_channels, connect, pi=pi, k=None, dropout=0.0, n_heads=self.n_heads)   
        self.conv3 = DynamicEdgeConv(hidden_channels, hidden_channels, k=3, dropout=0.1, n_heads=self.n_heads)
        self.fc2 = Sequential(
                            # Linear(hidden_channels, hidden_channels // 2),  
                            Linear(in_channels, hidden_channels // 2),
                            SELU(),
                            Linear(hidden_channels // 2, hidden_channels // 4),
                            SELU(),                 
                            Linear(hidden_channels // 4, out_channels)
                            ) 
        self.fc3 = Sequential(
                            # Linear(hidden_channels, hidden_channels * 2),  
                            Linear(in_channels, hidden_channels // 2),
                            SELU(),
                            Linear(hidden_channels * 2, hidden_channels * 4),
                            SELU(),                 
                            Linear(hidden_channels * 4, out_channels * l)
                            )               
    def forward(self, x,edge_index,edge_weight, batch, mask: Optional[torch.Tensor] = None):
        edge_index = knn_graph(x, 3, batch, loop=False) if edge_index is None else edge_index
        u = self.conv0(x, edge_index, edge_weight, batch)
        u = self.norm1(u, batch)
        # u = torch.cat((x, u), dim = 1)
        # edge_index = knn_graph(u, 3, batch, loop=False)
        # u = self.conv1(u, edge_index, edge_weight, batch)
        u = self.fc1(u)
        S = u.softmax(dim=-1)
        S = S.unsqueeze(0) if S.dim() == 2 else S
        x, _ = to_dense_batch(x, batch)  
        (batch_size, num_nodes, _) = x.size() 
        if mask is not None:
            mask = mask.view(batch_size, num_nodes, 1).to(x.dtype)
            x, S = x * mask, S * mask
        x = torch.matmul(S.transpose(1, 2), x)
        x = x.transpose(1, 2) / torch.sum(S, dim=1, keepdim=True).clamp(min=1.)
        # batch = torch.tensor(range(S.size(0)), dtype=torch.float32, device=S.device)
        batch = torch.tensor(range(S.size(0)), device=S.device)
        batch = batch.repeat_interleave(S.size(2))        
        x = x.view(-1, x.size(2)).t()
        S = S.view(-1, S.size(2))
        mu = x
        # x = self.conv2(x, _, batch)
        # y = self.conv3(x, _, batch)
        # z = x + y
        return u, mu, self.fc2(x), self.fc3(x), edge_index, batch


class STx_discriminator(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        torch.manual_seed(54321)
        self.conv = GINConv(Sequential(
                                        Linear(in_channels, hidden_channels),
                                        SELU(),
                                        Linear(hidden_channels, hidden_channels)
                            ))
        self.fc = Sequential(
                            Linear(hidden_channels, hidden_channels // 3),  
                            SELU(),
                            Linear(hidden_channels // 3, out_channels)                 
                            )
    def forward(self, z, edge_index, batch):        
        edge_index = knn_graph(z, 4, batch, loop=False) if edge_index is None else edge_index
        z = self.conv(z, edge_index)
        z = self.fc(z)
        return z
 
 
class STx_ARGA(GAE):
    def __init__(
        self,
        encoder: Module,
        discriminator: Module,
        decoder: Optional[Module] = None,
    ):
        super().__init__(encoder, decoder)
        self.discriminator = discriminator   
        self.criterion = CrossEntropyLoss()     
        reset(self.discriminator)   
    # def recon_loss(self, z: torch.Tensor,
    #             pos_edge_index: torch.Tensor, 
    #             pos_edge_weight: torch.Tensor,
    #             # pos_edge_weight: Optional[torch.Tensor] = None,
    #             neg_edge_index: torch.Tensor) -> torch.Tensor:
    #             # neg_edge_index: Optional[torch.Tensor] = None) -> torch.Tensor:
    #     pos_pred = self.decoder(z, pos_edge_index, sigmoid=True)
    #     if pos_edge_weight is None:
    #         pos_edge_weight = torch.ones_like(pos_pred, dtype=torch.float32)
    #     pos_loss = ((pos_edge_weight - pos_pred) ** 2).mean()
    #     if neg_edge_index is None:
    #         neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
    #     neg_pred = self.decoder(z, neg_edge_index, sigmoid=True)
    #     neg_loss = (neg_pred ** 2).mean() 
    #     return pos_loss + neg_loss                     
    def recon_loss(self, z: torch.Tensor, 
                pos_edge_index: torch.Tensor,   
                neg_edge_index: Optional[torch.Tensor] = None) -> torch.Tensor:
        pos_loss = -torch.log( self.decoder(z, pos_edge_index, sigmoid=True) + EPS).mean() 
        if neg_edge_index is None: 
            neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
            neg_edge_index, _ = remove_self_loops(neg_edge_index)
        neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid=True) + EPS).mean() 
        return pos_loss + neg_loss       
    def reset_parameters(self):
        super().reset_parameters()
        reset(self.discriminator)
    def reg_loss(self, z: torch.Tensor, edge_index: torch.Tensor, batch: torch.Tensor) -> torch.Tensor:
        real = torch.sigmoid(self.discriminator(z, edge_index, batch))
        real_loss = -torch.log(real + EPS).mean()
        return real_loss
    def discriminator_loss(self, z: torch.Tensor, edge_index: torch.Tensor, batch: torch.Tensor) -> torch.Tensor:
        real = torch.sigmoid(self.discriminator(torch.randn_like(z), edge_index, batch))
        fake = torch.sigmoid(self.discriminator(z.detach(), edge_index, batch))
        real_loss = -torch.log(real + EPS).mean()
        fake_loss = -torch.log(1 - fake + EPS).mean()
        return real_loss + fake_loss
 
class STx_ARGVA(STx_ARGA):
    def __init__(
        self,
        encoder: Module,
        discriminator: Module,
        decoder: Optional[Module] = None,
        l: Optional[torch.Tensor] = 1,
    ):
        super().__init__(encoder, discriminator, decoder) 
        self.l = l     
    def reparameterize(self, s: torch.Tensor, mu: torch.Tensor, A: torch.Tensor) -> torch.Tensor:
        # z = torch.matmul(s, mu)
        z = torch.matmul(s, mu.sigmoid())
        if self.training:
            C = torch.Tensor(onehot_to_label(s.t().cpu()).values).int().view(-1) 
            z = torch.cat([
                    torch.matmul(torch.randn(1, self.l, device=mu.device), A[C[i]].view(self.l, -1)) 
                        for i in range(s.size(0))
                    ]) + z
        return z    
    def encode(self, s: torch.Tensor, 
                    x: torch.Tensor, 
                    edge_index: torch.Tensor, 
                    edge_weight: torch.Tensor,
                    batch: torch.Tensor, 
                    mask: torch.Tensor) -> torch.Tensor:
        self.__u__, self.__m__, self.__v__, self.__A__, self.__e__, self.__b__ = self.encoder(x, edge_index, edge_weight, batch, mask)
        self.__A__ = self.__A__.clamp(min=MIN, max=MAX)
        self.__logstd__ =  torch.cat([
                        torch.matmul(self.__A__[i].view(self.l, -1).t(), self.__A__[i].view(self.l, -1)).diag().view(1, -1)
                            for i in range(self.__m__.size(0))
                        ]).log() 
        S = self.__u__.softmax(dim=-1)
        self.__adj__ = to_dense_adj(self.__e__)  
        self.__adj1__ = torch.matmul(torch.matmul(S.t(), self.__adj__), S)   
        s = S if s is None else s   
        z = self.reparameterize(s, self.__v__, self.__A__)
        return z, self.__b__ 
    def cluster_loss(self, u: torch.Tensor, y: Optional[torch.Tensor] = None) -> torch.Tensor:
        u = self.__u__ if u is None else u
        k = u.size(1)
        s = u.softmax(dim=-1)
        degrees = torch.einsum('ijk->ik', self.__adj__).t()
        m = torch.einsum('ij->', degrees)
        ca = torch.matmul(s.t(), degrees)
        cb = torch.matmul(degrees.t(), s)
        normalizer = torch.matmul(ca, cb) / 2 / m
        decompose = self.__adj1__ - normalizer
        spectral_loss = -_rank3_trace(decompose) / 2 / m
        spectral_loss = torch.mean(spectral_loss)
        # Orthogonal loss
        ss = torch.matmul(s.t(), s)
        i_s = torch.eye(k).type_as(ss)
        ortho_loss = torch.norm(
            ss / torch.norm(ss, dim=(-1, -2), keepdim=True) - i_s / torch.norm(i_s), dim=(-1, -2))
        ortho_loss = torch.mean(ortho_loss) 
        # Cluster loss:
        cluster_loss = torch.norm(torch.einsum('ij->i', ss)) / self.__adj__.size(1) * torch.norm(i_s) - 1  
        out = cluster_loss + ortho_loss + spectral_loss if y is None else cluster_loss + ortho_loss + spectral_loss + F.cross_entropy(u, y)        
        return out
    def kl_loss(
        self,
        mu: Optional[torch.Tensor] = None,
        logstd: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        mu = self.__v__ if mu is None else mu
        logstd = self.__logstd__ if logstd is None else logstd.clamp(max=MAX_LOGSTD)
        return -0.5 * torch.mean(
            torch.sum(1 + 2 * logstd - mu**2 - logstd.exp()**2, dim=1))
   

In [21]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics  import accuracy_score
import numpy as np

def cal_auc_ap_acc(sampling_n, edge_index,z,adj):
    auroc = []
    ap_score = []
    acc_score = []
    preds = []
    pos = []
    preds = []
    for src, dst in zip(edge_index[0], edge_index[1]):
        preds.append(adj[src, dst].item())
    preds = torch.tensor(preds).to(device)
    for i in range(sampling_n):
        neg_edge_index = negative_sampling(edge_index=edge_index, num_nodes=z.size(0))
        neg_edge_index, _ = remove_self_loops(neg_edge_index)
        preds_neg = []
        for src1, dst1 in zip(neg_edge_index[0], neg_edge_index[1]):
            preds_neg.append(adj[src1, dst1].item())
        preds_neg = torch.tensor(preds_neg).to(device)
        preds_all = torch.cat([preds, preds_neg],dim=0)
        labels_all = torch.cat([torch.ones(len(preds)), torch.zeros(len(preds_neg))])
        # # labels_all = torch.cat([torch.ones(len(preds_neg)), torch.zeros(len(preds))])
        labels_all = labels_all.detach().cpu()
        preds_all = preds_all.detach().cpu()
        roc_score = roc_auc_score(labels_all, preds_all)
        auroc.append(roc_score.item())
        ap = average_precision_score(labels_all, preds_all)
        ap_score.append(ap)
        acc = accuracy_score(labels_all, numpy.round(preds_all))
        acc_score.append(acc)
    return auroc,ap_score,acc_score

In [59]:
in_channels=N
hidden_channels=96
out_channels=16
m=7
l=3
K=3
connect=con
pi=0.75
n_heads=8
n_heads = n_heads
conv0 = SGWTConv(in_channels, hidden_channels, K).to(device)
norm1 = GraphNorm(hidden_channels).to(device)
conv1 = SGWTConv(in_channels + hidden_channels, hidden_channels, K).to(device)
fc1 = Sequential(
                            Linear(hidden_channels, m * 16),
                            SELU(),
                            Linear(m * 16, m * 4),
                            SELU(),
                            Linear(m * 4, m)
                            ).to(device)
conv2 = PPIEdgeConv(in_channels, hidden_channels, connect, pi=pi, k=None, dropout=0.0, n_heads=n_heads).to(device)   
conv3 = DynamicEdgeConv(hidden_channels, hidden_channels, k=3, dropout=0.1, n_heads=n_heads).to(device)
fc2 = Sequential(
                            Linear(hidden_channels, hidden_channels // 2),  
                            SELU(),
                            Linear(hidden_channels // 2, hidden_channels // 4),
                            SELU(),                 
                            Linear(hidden_channels // 4, out_channels)
                            ).to(device) 
fc3 = Sequential(
                            Linear(hidden_channels, hidden_channels * 2),  
                            SELU(),
                            Linear(hidden_channels * 2, hidden_channels * 4),
                            SELU(),                 
                            Linear(hidden_channels * 4, out_channels * l)
                            ).to(device)              


In [61]:
d[0]

Data(x=[1817, 1869], edge_index=[2, 5362], y=[1817], edge_weight=[5362], batch=[1817])

In [67]:
data = d[0].to(device)
x = data.x
src_mask = torch.bernoulli(torch.ones(data.x.size(0)) * 0.9).to(device)
# # x = (data.x+1).log().to(device)
# # z, _ = model.encode(None, data.x, data.edge_index,data.edge_weight, data.batch, src_mask)
edge_index =  data.edge_index
# edge_mask = torch.bernoulli(torch.ones(edge_index.shape[1]) * 0.9)
# indices_one = torch.nonzero(edge_mask, as_tuple=True)[0]
# edge_index = edge_index[:,indices_one]
# # edge_index = knn_graph(x, 3, batch, loop=False) if edge_index is None else edge_index
u = conv0(x, edge_index, None, data.batch)
# u = norm1(u, None)
# u = torch.cat((x, u), dim = 1)
# # # edge_index = knn_graph(u, 3, batch, loop=False)
# u = conv1(u, edge_index, None, None)
# u = fc1(u)
# S = u.softmax(dim=-1)
# S = S.unsqueeze(0) if S.dim() == 2 else S
# x, _ = to_dense_batch(x, None)  
# (batch_size, num_nodes, _) = x.size() 
# # if mask is not None:
# #     mask = mask.view(batch_size, num_nodes, 1).to(x.dtype)
# #     x, S = x * mask, S * mask
# x = torch.matmul(S.transpose(1, 2), x)
# x = x.transpose(1, 2) / torch.sum(S, dim=1, keepdim=True).clamp(min=1.)
# # # batch = torch.tensor(range(S.size(0)), dtype=torch.float32, device=S.device)
# batch = torch.tensor(range(S.size(0)), device=S.device)
# batch = batch.repeat_interleave(S.size(2))        
# x = x.view(-1, x.size(2)).t()
# S = S.view(-1, S.size(2))
# mu = x
# x = conv2(x, _, batch)
# print(x.shape)
# y = conv3(x, _, batch)
# print(y.shape)
# z = x + y
# return u, mu, self.fc2(z), self.fc3(z), edge_index, batch

RuntimeError: scatter(): Expected dtype int64 for index

In [22]:
def train(loader):
    model.train()
    l2_lambda = 0.001
    running_loss = 0
    print("train")
    # for data in loader: # Iterate in batches over the training dataset.
    data = loader.to(device)
    s = data.x.shape[0]
    src_mask = torch.bernoulli(torch.ones(data.x.size(0)) * 0.9).to(device)
    # x = (data.x+1).log().to(device)
    # z, _ = model.encode(None, data.x, data.edge_index,data.edge_weight, data.batch, src_mask)
    edge_index =  data.edge_index
    edge_mask = torch.bernoulli(torch.ones(edge_index.shape[1]) * 0.9)
    indices_one = torch.nonzero(edge_mask, as_tuple=True)[0]
    edge_index = edge_index[:,indices_one]
    z, _ = model.encode(None, data.x, edge_index,None, None, src_mask)
    w = model.__u__
    cl = w.cpu().softmax(dim=-1)
    for _ in range(5):
        discriminator.train()
        discriminator_optimizer.zero_grad()
        discriminator_loss = model.discriminator_loss(z, edge_index, None)
        discriminator_loss.backward(retain_graph=True)
        discriminator_optimizer.step()
    loss = 0
    clast_loss = 0
    loss = loss + model.reg_loss(z, None, None)
    loss = loss + model.recon_loss(z, edge_index, None)
    loss = loss + model.cluster_loss(None, data.y)
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    reg_loss =  model.reg_loss(z, None, None)
    recon_loss = model.recon_loss(z, edge_index,None)
    clast_loss = model.cluster_loss(None, data.y)
    l2_regularization = torch.tensor(0., requires_grad=True, device=device)
    for name, param in model.encoder.named_parameters():
        if 'bias' not in name:
            l2_regularization = l2_regularization + torch.norm(param, p=1)
    loss = loss + l2_lambda * l2_regularization
    clip_grad_value_(model.encoder.parameters(), .1)
    encoder_optimizer.step()
    encoder_optimizer.zero_grad()
    running_loss += loss.item()
    loss.backward()
    ari1 = adjusted_rand_score(onehot_to_label(cl.t()).values.reshape(-1), data.y.cpu())
    acc1 = accuracy_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    nmi1 =  normalized_mutual_info_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    adj = model.decoder.forward_all(z)
    auroc,ap_score,acc_score = cal_auc_ap_acc(1,data.edge_index,z,adj)
    auc = np.mean(auroc)
    ap = np.mean(ap_score)
    acc = np.mean(acc_score)
    f.write( ','.join(map(str, torch.tensor([epoch, loss, reg_loss, recon_loss, clast_loss,ari1, acc1, nmi1,auc,ap,acc,s]).numpy())))
    f.write('\n')
    print((f'Epoch: {epoch:03d}, Loss: {loss:.3f}, loss reg: {reg_loss:.3f}, '
            f'loss recon: {recon_loss:.3f}, '
        f'loss class: {clast_loss:.3f}, '
        f'ARI: {ari1:.3f},'
        f'ACC: {acc1:.3f},'
        f'NMI: {nmi1:.3f},'
        f'AUC: {auc:.3f},'
        f'AP: {ap:.3f},'
        f'ACC1: {acc:.3f},'
        f'Shape: {s}'))
    encoder_optimizer.step()
    encoder_optimizer.zero_grad()
    running_loss += loss.item()
    return running_loss / len(loader)


In [23]:
@torch.no_grad()
def test(loader):
    print("test")
    model.eval()
    loss_reg = 0
    loss_clust = 0
    loss_recon = 0
    loss_recon1 = 0
    loss_clust1 = 0
    # for data in loader: # Iterate in batches over the training/test dataset.
    data = loader.to(device)
    # x = (data.x+1).log().to(device)
    # z, _ = model.encode(None, data.x, data.edge_index,None, data.batch, None)
    z, _ = model.encode(None, data.x, data.edge_index,None, None, None)
    loss_reg =  model.reg_loss(z, None, None)
    loss_recon = model.recon_loss(z, data.edge_index, None)
    loss_clust =  model.cluster_loss(None, data.y)
    # loss_clust = 0
    # loss_recon1 = loss_recon1 + model.recon_loss(z, data.edge_index, None)
    loss_clust1 = loss_clust1 + model.recon_loss(z, data.edge_index, None)
    w = model.__u__
    cl = w.cpu().softmax(dim=-1)
    s = data.x.shape[0]
    ari = adjusted_rand_score(onehot_to_label(cl.t()).values.reshape(-1), data.y.cpu())
    acc = accuracy_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    nmi =  normalized_mutual_info_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    # ami = adjusted_mutual_info_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    # print(ari)
    # loss = 0
    adj = model.decoder.forward_all(z)
    auroc,ap_score,acc_score = cal_auc_ap_acc(1,data.edge_index,z,adj)
    auc = np.mean(auroc)
    ap = np.mean(ap_score)
    acc1 = np.mean(acc_score)
    f1.write( ','.join(map(str, torch.tensor([epoch, loss, loss_reg,loss_recon, loss_clust,ari,acc,nmi,auc,ap,acc,s]).numpy())))
    f1.write('\n')
        # print("test")
    print((f'Epoch: {epoch:03d}, Loss: {loss:.3f}, loss reg: {loss_reg:.3f}, '
            f'loss recon: {loss_recon:.3f}, '
        f'loss class: {loss_clust:.3f}, '
        f'ARI: {ari:.3f},'
        f'ACC: {acc:.3f},'
        f'NMI: {nmi:.3f},'
        f'AUC: {auc:.3f},'
        f'AP: {ap:.3f},'
        f'ACC1: {acc1:.3f},'
        f'Shape: {s}'))
    return loss_reg , loss_clust, loss_recon,auc # Derive ratio of correct predictions.


In [153]:
encoder = STx_encoder(in_channels=N,
              hidden_channels=96,
              out_channels=16,
              m=7, l=3, K=3,
              connect=con, pi=0.75, n_heads=8)
discriminator = STx_discriminator(in_channels=16, hidden_channels=8, out_channels=1)
model = STx_ARGVA(encoder, discriminator, l=3)
model = model.to(device)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1E-3)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=1E-4)

In [154]:
data = d[0].to(device)
s = data.x.shape[0]
src_mask = torch.bernoulli(torch.ones(data.x.size(0)) * 0.9).to(device)
# x = (data.x+1).log().to(device)
# z, _ = model.encode(None, data.x, data.edge_index,data.edge_weight, data.batch, src_mask)
edge_index =  data.edge_index
edge_mask = torch.bernoulli(torch.ones(edge_index.shape[1]) * 0.9)
indices_one = torch.nonzero(edge_mask, as_tuple=True)[0]
edge_index = edge_index[:,indices_one]
# z, _ = model.encode(None, data.x, edge_index,None, data.batch, src_mask)

In [36]:
def create_mask(num_nodes, train_ratio=0.4):
    mask = torch.rand(num_nodes) < train_ratio
    return mask

In [44]:
sample_id = ["151673"]
d = SGEDataset(raw_dir='/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/raw',id=sample_id)

sample ID: 151673


In [15]:

encoder = STx_encoder(in_channels=N,
              hidden_channels=96,
              out_channels=16,
              m=7, l=3, K=3,
              connect=con, pi=0.75, n_heads=8)
discriminator = STx_discriminator(in_channels=16, hidden_channels=8, out_channels=1)
model = STx_ARGVA(encoder, discriminator, l=3)
model = model.to(device)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1E-3)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=1E-4)

model1 = model
model1.load_state_dict(torch.load("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/mask/best_model_edge_mask1_73.pkl"))
model1.eval()

model = model1

In [24]:
f = open(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/edge_index/train.csv', 'w')
f1 = open(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/edge_index/test.csv', 'w')
f.writelines(['epoch,','loss,', 'reg loss,', 'recon_loss,', 'class loss,',  'ari,','acc,','nmi,', 'auc,','ap,','acc1,','sample\n'])
f1.writelines(['epoch,','loss,', 'reg loss,',  'recon_loss,', 'class loss,','ari,','acc,','nmi,', 'auc,','ap,','acc1,','sample\n'])
encoder = STx_encoder(in_channels=N,
                hidden_channels=96,
                out_channels=16,
                m=7, l=3, K=3,
                connect=con, pi=0.75, n_heads=8)
discriminator = STx_discriminator(in_channels=16, hidden_channels=8, out_channels=1)
model = STx_ARGVA(encoder, discriminator, l=3)
model = model.to(device)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1E-2)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=1E-3)
best_value = 0
for epoch in range(1, 2000):
    loss = train(d[0])
    loss_reg, loss_recon, loss_clust, auc = test(d[1])
    if(auc > best_value):
        print(f"Now best model is epoch{epoch}")
        best_value = auc
        best_model = model.state_dict()
        torch.save(best_model, os.path.join(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/edge_index/',rf'best_model_edge.pkl'))

train


RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x48 and 192x384)

In [54]:
f = open(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/Li_0515/train_con_73.csv', 'w')
f1 = open(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/Li_0515/test_con_73.csv', 'w')
f.writelines(['epoch,','loss,', 'reg loss,', 'recon_loss,', 'class loss,',  'ari,','acc,','nmi,', 'auc,','ap,','acc1,','sample\n'])
f1.writelines(['epoch,','loss,', 'reg loss,',  'recon_loss,', 'class loss,','ari,','acc,','nmi,', 'auc,','ap,','acc1,','sample\n'])
encoder = STx_encoder(in_channels=N,
              hidden_channels=96,
              out_channels=16,
              m=7, l=3, K=8,
              connect=con, pi=0.75, n_heads=8)
discriminator = STx_discriminator(in_channels=16, hidden_channels=8, out_channels=1)
model = STx_ARGVA(encoder, discriminator, l=3)
model = model.to(device)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1E-3)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=1E-4)
best_value = 0
for epoch in range(1, 1000):
    loss = train(d[0])
    loss_reg, loss_recon, loss_clust, auc = test(d[1])
    if(auc > best_value):
        print(f"Now best model is epoch{epoch}")
        best_value = auc
        best_model = model.state_dict()
        torch.save(best_model, os.path.join(r'/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/Li_0515/',rf'best_model_edge_con_73.pkl'))

train


RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x48 and 192x384)

In [22]:
encoder = STx_encoder(in_channels=N, 
             hidden_channels=96, 
             out_channels=32, 
             m=7, l=2, K=8,
             connect=con, pi=0.75, n_heads=8)
discriminator = STx_discriminator(in_channels=32, hidden_channels=8, out_channels=1)
model = STx_ARGVA(encoder, discriminator, l=2)
model = model.to(device)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=1E-4)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=1E-5)


In [23]:
f = open(r'argva0516.csv', 'w')
f.writelines(['train loss,', 'test reg loss,', 'test recon loss,', 'test clust-loss\n'])
f.close()
#f = open(r'argva0516.csv', 'a')
for epoch in range(1, 501):
    loss = train(d[0])
    loss_reg, loss_recon, loss_clust = test(d[1])
    f.write( ','.join(map(str, torch.tensor([loss, loss_reg, loss_recon, loss_clust]).numpy())))
    f.write('\n')
    print((f'Epoch: {epoch:03d}, Loss: {loss:.3f}, Test loss reg: {loss_reg:.3f}, '
           f'Test loss recon: {loss_recon:.3f}, Test clust-loss: {loss_clust:.3f}')) 

TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [16]:
for data in d:
    data = data.to(device)
    z, _ = model.encode(None, data.x, data.edge_index,None, None, None)
    w = model.__u__
    cl = w.cpu().softmax(dim=-1)
    s = data.x.shape[0]
    ari = adjusted_rand_score(onehot_to_label(cl.t()).values.reshape(-1), data.y.cpu())
    acc = accuracy_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    nmi =  normalized_mutual_info_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    # ami = adjusted_mutual_info_score(data.y.cpu(), onehot_to_label(cl.t()).values.reshape(-1))
    # print(ari)
    # loss = 0
    adj = model.decoder.forward_all(z)
    auroc,ap_score,acc_score = cal_auc_ap_acc(1,data.edge_index,z,adj)
    auc = np.mean(auroc)
    ap = np.mean(ap_score)
    acc1 = np.mean(acc_score)
    print("ari:",ari)
    print("acc:",acc)
    print("nmi:",nmi)
    print("auc:",auc)
    print("ap:",ap)
    print("acc1:",acc1)


ari: 0.6696482423793315
acc: 0.8274946921443737
nmi: 0.7148373106616541
auc: 0.8846254823364205
ap: 0.8491499006955852
acc1: 0.7717816500711238
ari: 0.6486621723492817
acc: 0.8199189345686161
nmi: 0.7076036215198002
auc: 0.8984042839701235
ap: 0.8635687305466829
acc1: 0.8167372881355932


In [17]:
import torch
indices_k = model.encoder.conv2.knet.state_dict()['0.indices']
indices_q = model.encoder.conv2.qnet.state_dict()['0.indices']
weight_k = model.encoder.conv2.knet.state_dict()['0.weights']
weight_q = model.encoder.conv2.qnet.state_dict()['0.weights']



# 初始化 n x 2n 矩阵
n = con[0].max()+ 1  # 基于 con 中的最大索引计算 n
matrix_q = torch.zeros((n, 2*n))

# 填充权重
for i in range(indices_q.shape[1]):
    start, end = indices_q[:, i]
    matrix_q[start, end] = weight_q[i]
# 填充权重值
matrix_k = torch.zeros((n, 2*n))
for i in range(indices_k.shape[1]):
    start, end = indices_k[:, i]
    matrix_k[start, end] = weight_k[i]
# 此时 matrix 为根据连接和权重构建的 n x 2n 矩阵


In [18]:
w = torch.matmul(matrix_k.T,matrix_q)

In [19]:
k = 300
values, flat_indices = w.view(-1).topk(k)
indices_2d = torch.stack((flat_indices // w.size(1), flat_indices % w.size(1)))
indices_2d



import pandas as pd
indices = pd.DataFrame(indices_2d.detach().cpu().T)
indices['values'] = values.detach().cpu()
indices.columns = ["row1","row2","values"]

In [22]:
ppi.columns = ["0","row1","row2"]

In [23]:
con_df = pd.DataFrame(con.detach().cpu().T)
con_df.columns = ["row1","row2"]

result = pd.merge(indices,con_df, on=['row1', 'row2'],how = "left")
indices.columns = ["row1","row2","values"]

ppi1 = ppi
ppi1[['Protein1', 'Protein2']] = ppi1.iloc[:, 0].str.split('-', expand=True,n=1)

df_row1_protein1 = ppi1[['row1', 'Protein1']].rename(columns={'row1': 'row', 'Protein1': 'Protein'})
df_row2_protein2 = ppi1[['row2', 'Protein2']].rename(columns={'row2': 'row', 'Protein2': 'Protein'})

new_df = pd.concat([df_row1_protein1, df_row2_protein2], ignore_index=True)
new_df.drop_duplicates()

row_to_protein = pd.Series(new_df.Protein.values,index=new_df.row).to_dict()
result['Protein1'] = result['row1'].map(row_to_protein)
result['Protein2'] = result['row2'].map(row_to_protein)



In [25]:
result.to_csv("/shareN/data8/SwapTmp/fy/Spatial/ARGA-ARVGA/SGE2/result/mask/result_73.csv")